# Bone surface mesh pre-processing
- This notebook cleans and remeshes the raw output of 3D Slicer
- You can skip this notebook if you already have high quality bone surfaces meshes

## imports and user-defined properties

In [1]:
import numpy as np
import meshplot as mp
import os
import igl
from pathlib import Path
import sys

sys.path.append(str(Path.home()/'Documents'/'Github'/'libhip'))
sys.path.append('../')

import src


INPUT SUBJECT INFORMATION:
- subject_id = choose between m1 -> m11 to select one of the detaset in the repository.
- i_dim, o_dim: the input and output dimension ( 'mm' = millimeters, 'm' = meters )
- o_format: the format you want the files to be save at ( '.obj' , '.stl' )

In [2]:
# subject id
subject_id = 'm1'  

# dimensions 
i_dim  = 'mm'     
o_dim  = 'mm'

# file format and suffix
i_format = '.stl'
o_format = '.obj'

DIRECTORIES and PATHS:
- main_dir: locate this directory to where the libhip repository is cloned to.
- i_dir:  the directory of the input bone surface mesh files.
- o_dir:  the directory of the cleaned and remeshed bone model results.

In [3]:
# input and output directories
main_dir = Path('..')

i_dir = main_dir / 'model_repository'/ 'slicer_raw_output'/ subject_id
o_dir = main_dir / 'model_generation'/ 'preprocessing_output'/ subject_id

config_name  = subject_id + '_config'
config_path  = str((main_dir/ 'config'/ config_name).with_suffix('.yml'))
config       = src.Config (config_path)

# Remove all files inside output directory if it exists, otherwise create it
if o_dir.is_dir():
    for file in o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    o_dir.mkdir(exist_ok=False)

In [4]:
# input paths
input_sacrum  = subject_id + '_sacrum_'+ i_dim
input_lhip    = subject_id + '_lhip_'+ i_dim
input_rhip    = subject_id + '_rhip_'+ i_dim
input_lfemur  = subject_id + '_lfemur_'+ i_dim
input_rfemur  = subject_id + '_rfemur_'+ i_dim

input_sacrum_path  = str((i_dir /input_sacrum).with_suffix(i_format))
input_lhip_path    = str((i_dir /input_lhip).with_suffix(i_format))
input_rhip_path    = str((i_dir /input_rhip).with_suffix(i_format))
input_lfemur_path  = str((i_dir /input_lfemur).with_suffix(i_format))
input_rfemur_path  = str((i_dir /input_rfemur).with_suffix(i_format))

# output paths
output_sacrum  = subject_id + '_remeshed_sacrum_'+ o_dim
output_lhip    = subject_id + '_remeshed_lhip_'+ o_dim
output_rhip    = subject_id + '_remeshed_rhip_'+ o_dim
output_lfemur  = subject_id + '_remeshed_lfemur_'+ o_dim
output_rfemur  = subject_id + '_remeshed_rfemur_'+ o_dim

output_sacrum_path  = str((o_dir /output_sacrum).with_suffix(o_format))
output_lhip_path    = str((o_dir /output_lhip).with_suffix(o_format))
output_rhip_path    = str((o_dir /output_rhip).with_suffix(o_format))
output_lfemur_path  = str((o_dir /output_lfemur).with_suffix(o_format))
output_rfemur_path  = str((o_dir /output_rfemur).with_suffix(o_format))

REMESHING PARAMS:
- eps: the envelope of size epsilon. Using smaller envelope preserves features better but also takes longer time. 
  The default value of epsilon is b/1000, where b is the length of the diagonal of the bounding box. 
  Please refer to https://github.com/wildmeshing/fTetWild for more information.
- l_* : the ideal edge length. Using smaller ideal edge length gives a denser mesh but also takes longer time. 
  The default ideal edge length is b/20.
  Please refer to https://github.com/wildmeshing/fTetWild for more information.

In [5]:
config_remesh = config.mesh_var

# the envelope of size epsilon
eps      = config_remesh.remesh_epsilon

# edge length for the femur bones
l_leg    = config_remesh.remesh_edge_length_leg

# edge length for the the sacrum and the hip bones
l_girdle = config_remesh.remesh_edge_length_girdle


## implementation

### 1- read and clean up bone models
This function first reads an input surface mesh as a set vertices and faces. Next, the duplicated faces and unreferenced vertices are removed from the input bone surface mesh.
- The cleaned models are saved as a set of '_vertices' and '_faces'.
- The number of faces after cleaning is printed out for each bone model.


In [6]:
s1_vertices, s1_faces = src.read_and_clean (input_sacrum_path, i_dim ) 
s2_vertices, s2_faces = src.read_and_clean (input_lhip_path, i_dim )
s3_vertices, s3_faces = src.read_and_clean (input_rhip_path, i_dim ) 
s4_vertices, s4_faces = src.read_and_clean (input_lfemur_path, i_dim )
s5_vertices, s5_faces = src.read_and_clean (input_rfemur_path, i_dim ) 

number of faces after cleaning 164868
number of faces after cleaning 207040
number of faces after cleaning 206876
number of faces after cleaning 100796
number of faces after cleaning 100472


In [7]:
frame = mp.plot(s1_vertices, s1_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (s2_vertices, s2_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (s3_vertices, s3_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (s4_vertices, s4_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (s5_vertices, s5_faces, c = src.bone, shading = src.sh_true)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.021293…

4

### 2- remesh the bone models
We remesh the bone surface mesh to improve the quality and resize the triangles. We use fTetWild to mesh the volume of the surface boundary and re-extract the surface of the volume mesh as the resulting remeshed surface. 
- This meshing method allows vertices of the boundary of the volumetric mesh to move within an epsilon (eps) envelope of the winding number field level-set that defines the surface geometry.
- The number of faces after remeshing is printed out for each bone model.


In [ ]:
rm_s1_vertices, rm_s1_faces = src.remesh (s1_vertices, s1_faces, eps, l_girdle)
rm_s2_vertices, rm_s2_faces = src.remesh (s2_vertices, s2_faces, eps, l_girdle)
rm_s3_vertices, rm_s3_faces = src.remesh (s3_vertices, s3_faces, eps, l_girdle)
rm_s4_vertices, rm_s4_faces = src.remesh (s4_vertices, s4_faces, eps, l_leg)
rm_s5_vertices, rm_s5_faces = src.remesh (s5_vertices, s5_faces, eps, l_leg)

In [ ]:
frame = mp.plot(rm_s1_vertices, rm_s1_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_s2_vertices, rm_s2_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_s3_vertices, rm_s3_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_s4_vertices, rm_s4_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rm_s5_vertices, rm_s5_faces, c = src.bone, shading = src.sh_true)

### 3- export results
In this step we save the cleaned bone models suface meshes in '.obj' file format. The output dimension and path is defined by the user.


In [ ]:
src.save_surface (rm_s1_vertices, rm_s1_faces, o_dim, output_sacrum_path )
src.save_surface (rm_s2_vertices, rm_s2_faces, o_dim, output_lhip_path)
src.save_surface (rm_s3_vertices, rm_s3_faces, o_dim, output_rhip_path)
src.save_surface (rm_s4_vertices, rm_s4_faces, o_dim, output_lfemur_path )
src.save_surface (rm_s5_vertices, rm_s5_faces, o_dim, output_rfemur_path )